# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [2]:
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add

Using TensorFlow backend.


##### 定義 Separable Convolution 函數 (請在此實做)

In [3]:
def InvertedRes(input, expansion):
    # inverted residual with linear bottleneck : 升維 -> depthwise convolution -> 降維

    # 1.升維(expansion layer), expansion 因子爲 6
    x = Conv2D(filters = expansion*3, kernel_size=(1,1), padding='same')(input)  # conv layer
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # 2.depthwise convolution :
    x = DepthwiseConv2D(kernel_size=(3,3), padding='same')(x)  # conv layer
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # 3.pointwise convolution : 降維
    x = Conv2D(filters=3, kernel_size=(1,1), padding='same')(x)  # conv layer
    x = BatchNormalization()(x)
    #x = ReLU()(x)  # bottleneck, Don't use ReLU() activation
    
    x = Add()([x, input])
    
    return x

##### 建構模型

In [4]:
input = Input((64, 64, 3))
output = InvertedRes(input, 6)
model = Model(inputs=input, outputs=output)
model.summary()












__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 18)   72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 64, 64, 18)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________